# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,caravelas,-17.7125,-39.2481,75.272,91,14,1.92,BR,1674961372
1,1,sitka,57.0531,-135.3300,35.492,64,20,2.06,US,1674961372
2,2,lebu,-37.6167,-73.6500,60.476,91,0,8.05,CL,1674961373
3,3,ponta do sol,32.6667,-17.1000,57.002,79,42,3.07,PT,1674961373
4,4,tete,-16.1564,33.5867,75.290,100,20,0.00,MZ,1674961374


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 600,
    frame_height = 400,
    scale = 2,
    hover_cols = ['City','Max Temp','Humidity','Cloudiness','Wind Speed']
)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Humidity,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
city_limits = city_data_df.loc[(city_data_df['Max Temp'] > 60) & (city_data_df['Max Temp'] < 80)]
city_limits = city_limits.loc[city_limits['Humidity'] < 50]
city_limits = city_limits.loc[city_limits['Cloudiness'] < 20]
city_limits = city_limits.loc[city_limits['Wind Speed'] < 1.5]
# Drop any rows with null values
city_limits = city_limits.dropna()

# Display sample data
city_limits

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,new norfolk,-42.7826,147.0587,75.704,32,0,0.89,AU,1674961402
219,219,phonhong,18.5007,102.4166,65.912,43,1,1.34,LA,1674961560
221,221,tura,25.5198,90.2201,71.636,44,0,0.47,IN,1674961562
338,338,along,28.1667,94.7667,67.010,48,2,0.27,IN,1674961675
396,396,los andes,-32.8337,-70.5983,71.222,36,0,0.63,CL,1674961727
543,543,ramanuj ganj,23.8000,83.7000,63.662,38,0,1.22,IN,1674961876


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_limits.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,32,new norfolk,-42.7826,147.0587,75.704,32,0,0.89,AU,1674961402,
219,219,phonhong,18.5007,102.4166,65.912,43,1,1.34,LA,1674961560,
221,221,tura,25.5198,90.2201,71.636,44,0,0.47,IN,1674961562,
338,338,along,28.1667,94.7667,67.010,48,2,0.27,IN,1674961675,
396,396,los andes,-32.8337,-70.5983,71.222,36,0,0.63,CL,1674961727,
543,543,ramanuj ganj,23.8000,83.7000,63.662,38,0,1.22,IN,1674961876,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lon = hotel_df.loc[index]['Lng']
    lat = hotel_df.loc[index]['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
new norfolk - nearest hotel: Woodbridge on the Derwent
phonhong - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
along - nearest hotel: Hotel Toshi Palace
los andes - nearest hotel: Hotel Gloria
ramanuj ganj - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,32,new norfolk,-42.7826,147.0587,75.704,32,0,0.89,AU,1674961402,Woodbridge on the Derwent
219,219,phonhong,18.5007,102.4166,65.912,43,1,1.34,LA,1674961560,No hotel found
221,221,tura,25.5198,90.2201,71.636,44,0,0.47,IN,1674961562,No hotel found
338,338,along,28.1667,94.7667,67.010,48,2,0.27,IN,1674961675,Hotel Toshi Palace
396,396,los andes,-32.8337,-70.5983,71.222,36,0,0.63,CL,1674961727,Hotel Gloria
543,543,ramanuj ganj,23.8000,83.7000,63.662,38,0,1.22,IN,1674961876,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
cities_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 600,
    frame_height = 400,
    scale = 2,
    hover_cols = ['City','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Hotel Name']
)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name)